In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
import re
import json

import numpy as np
import pandas as pd
from tqdm import tqdm

# !pip install konlpy
from konlpy.tag import Okt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 57.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.6/465.6 KB 45.3 MB/s eta 0:00:00


In [38]:
FILTERS = "([~.,!?\"':;)(])"

PAD = "<PAD>"
STD = "<STD>"
END = "<END>"
UNK = "<UNK>"

# 어떤 의미도 없는 패딩 토큰
PAD_INDEX = 0

# 시작 토큰
STD_INDEX = 1

# 종료 토큰
END_INDEX = 2

# 사전에 없는 단어
UNK_INDEX = 3

MARKER = [PAD,STD,END,UNK]
CHANGE_FILTER = re.compile(FILTERS)

# 데이터 분석을 통해 문장 최대 길이를 설정함
# 문장 최대 길이 25
MAX_SQEUENCE = 25 

In [39]:

# 질문데이터 list, 응답데이터 list 반환하는 메서드
def load_data(path):
    # 판다스를 통해서 데이터를 불러온다.
    
    # * header=0 첫 인덱스를 0으로 시작함

    data_df = pd.read_csv(path, header=0,encoding='cp949')
    # 질문과 답변 열을 가져와 question과 answer에 넣는다.
    question, answer = list(data_df['Q']), list(data_df['A'])

    return question, answer

# data = 질문 데이터 list , 대답 데이터 list 모두 더한 뒤 extend
# data = [형태소로 띄어쓰기 된 질문 문장1,....형태소로 띄어쓰기 된 대답 문장1.....]
def data_tokenizer(data):
    # 토크나이징 해서 담을 배열 생성
    words = []
    
    # sentence는 문장 1개
    for sentence in data:
        # FILTERS = "([~.,!?\"':;)(])"
        # 위 필터와 같은 값들을 정규화 표현식을
        # 통해서 모두 "" 으로 변환 해주는 부분이다.
        sentence = re.sub(CHANGE_FILTER, "", sentence)

        # sentence는 띄어쓰기로 형태소 구분된 문장임
        # 띄어쓰기된 단어 하나하나 넣어줌
        for word in sentence.split():
            words.append(word)
    # 토그나이징과 정규표현식을 통해 만들어진
    # 값들을 넘겨 준다.

    # 형태소 단위의 단어 리스트를 반환함
    return [word for word in words if word]


def prepro_like_morphlized(data):
    morph_analyzer = Okt()
    result_data = list()

     # data 는 [문장1, 문장2...]
     # seq는 문장1개
    for seq in tqdm(data):

        # 문장의 띄어쓰기 없애고
        # 형태소 분석
        # okt().morphs 하면 
        # ['모바일', '게임', '은', '재밌다', '열심히', '해서', '만', '랩', '을', '찍어야지', '~', 'ㅎㅎㅎ']으로 반환

        # "",join() 함수 예제
        # list를 string 으로
        # >>> time_list
        # ['10', '34', '17']
        # >>> ':'.join(time_list)
        # '10:34:17'

        # morphlized_seq = 띄어쓰기로 형태소 구분된 문자열 
        morphlized_seq = " ".join(morph_analyzer.morphs(seq.replace(' ', '')))

        result_data.append(morphlized_seq)

    # result_data = 띄어쓰기로 형태소 구분된 문자열의 리스트
    print(f'prepro_like_morphlized 메서드 결과 result_data[0]:{result_data[0]}')
    return result_data


def load_vocabulary(path, vocab_path, tokenize_as_morph=False):
    # 사전을 담을 배열 준비한다.
    vocabulary_list = []
    # 사전을 구성한 후 파일로 저장 진행한다.
    # 그 파일의 존재 유무를 확인한다.
    if not os.path.exists(vocab_path):
        # 이미 생성된 사전 파일이 존재하지 않으므로
        # 데이터를 가지고 만들어야 한다.
        # 그래서 데이터가 존재 하면 사전을 만들기 위해서
        # 데이터 파일의 존재 유무를 확인한다.
        if (os.path.exists(path)):
            # 데이터가 존재하니 판단스를 통해서
            # 데이터를 불러오자
            data_df = pd.read_csv(path, encoding='cp949')
            # 판다스의 데이터 프레임을 통해서
            # 질문과 답에 대한 열을 가져 온다.
            question, answer = list(data_df['Q']), list(data_df['A'])

            # question = [문장1, 문장2...]
            # answer = [문장1, 문장2...]

            if tokenize_as_morph:  # 형태소에 따른 토크나이져 처리
                question = prepro_like_morphlized(question)
                answer = prepro_like_morphlized(answer)
            data = []

            # 질문과 답변을 extend을
            # 통해서 구조가 없는 배열로 만든다.(1차원 리스트로 저장)
            # list.extend 예제
            # https://m.blog.naver.com/wideeyed/221541104629

            data.extend(question)
            data.extend(answer)
            # 토큰나이져 처리 하는 부분이다.
            words = data_tokenizer(data)
            # 공통적인 단어에 대해서는 모두
            # 필요 없으므로 한개로 만들어 주기 위해서
            # set해주고 이것들을 리스트로 만들어 준다.
            words = list(set(words))
            # 데이터 없는 내용중에 MARKER를 사전에
            # 추가 하기 위해서 아래와 같이 처리 한다.
            # 아래는 MARKER 값이며 리스트의 첫번째 부터
            # 순서대로 넣기 위해서 인덱스 0에 추가한다.
            # PAD = "<PADDING>"
            # STD = "<START>"
            # END = "<END>"
            # UNK = "<UNKNWON>"
            words[:0] = MARKER
        # 사전을 리스트로 만들었으니 이 내용을
        # 사전 파일을 만들어 넣는다.
        with open(vocab_path, 'w', encoding='utf-8') as vocabulary_file:

            # words는 중복없는 형태소 단위의 단어 리스트임(질문+응답 데이터 모두 포함)
            for word in words:
                vocabulary_file.write(word + '\n')
        # vocaulbary_file 예시
        # 나
        # 는
        # 밥
        # 을
        # 좋아
        # 한다

    # 사전 파일이 존재하면 여기에서
    # 그 파일을 불러서 배열에 넣어 준다.
    with open(vocab_path, 'r', encoding='utf-8') as vocabulary_file:
        for line in vocabulary_file:
            vocabulary_list.append(line.strip())

    # vocabulary_list는 [형태소 단위의 중복 없는 단어1,2,....]

    # 배열에 내용을 키와 값이 있는
    # 딕셔너리 구조로 만든다.
    char2idx, idx2char = make_vocabulary(vocabulary_list)
    # 두가지 형태의 키와 값이 있는 형태를 리턴한다.

    # 반환 => 단어: 인덱스 , 인덱스: 단어, 총 단어의 갯수
    return char2idx, idx2char, len(char2idx)


def make_vocabulary(vocabulary_list):
    # 리스트를 키가 단어이고 값이 인덱스인
    # 딕셔너리를 만든다.
    char2idx = {char: idx for idx, char in enumerate(vocabulary_list)}
    # 리스트를 키가 인덱스이고 값이 단어인
    # 딕셔너리를 만든다.
    idx2char = {idx: char for idx, char in enumerate(vocabulary_list)}
    # 두개의 딕셔너리를 넘겨 준다.

    # char2idx = 단어:인덱스 딕셔너리
    # idx2char = 인덱스:단어 딕셔너리
    return char2idx, idx2char

PATH = '/content/drive/MyDrive/새싹_인공지능SW교육/프로젝트/새싹_최종프로젝트/chatbot_s2s/data/ChatbotData.csv'
VOCAB_PATH = '/content/drive/MyDrive/새싹_인공지능SW교육/프로젝트/새싹_최종프로젝트/chatbot_s2s/data/vocabulary.txt'
inputs , outputs = load_data(PATH)
char2idx, idx2char, vocab_size = load_vocabulary(PATH, VOCAB_PATH, tokenize_as_morph=False)

In [40]:
# inputs = 질문 리스트, outputs = 대답 리스트

inputs[1], outputs[1]

('1지망 학교 떨어졌어', '위로해 드립니다.')

In [41]:
# char2dix 는 형태소 마다 인덱스를 부여했음
char2idx['나'] ,char2idx['하세요'],char2idx['어제'],char2idx['하다'] , char2idx['너는'] , char2idx['누구야'] , idx2char[13], vocab_size 

(16114, 1931, 10104, 3615, 3863, 11004, '거니', 20705)

In [42]:
char2idx['나'] ,char2idx['나는'],char2idx['어제'],char2idx['어제는'] , char2idx['먹었다'] , char2idx['먹다']

(16114, 1391, 10104, 8357, 14039, 960)

In [43]:
char2idx['저'] ,char2idx['저는'],char2idx['어제는'],char2idx['보름'] , char2idx['아빠'] , char2idx['내가']

(14701, 3611, 8357, 18470, 9931, 11178)

In [44]:
char2idx['<PAD>'], char2idx['<STD>'],char2idx['<END>'],char2idx['<UNK>'], idx2char[0], idx2char[1], idx2char[2], idx2char[3]

(0, 1, 2, 3, '<PAD>', '<STD>', '<END>', '<UNK>')

In [45]:
# okt가 형태소 분석을 100% 정확하게 했다고 볼 수 없다

# 여기까지 dict형 형태소 사전 만듬

# 1. 형태소:인덱스
# 2. 인덱스:형태소
# 3. 단어갯수

In [47]:
MAX_SQEUENCE = 25
MAX_SEQUENCE = 25 

In [50]:
# Main문에서 
# enc_processing(inputs, char2idx, tokenize_as_morph=False)
# enc_processing(질문 리스트, 헝태소:인덱스 dict, tokenize_as_morph=False)
def enc_processing(value, dictionary, tokenize_as_morph=False):

    MAX_SEQUENCE = 25 
  
    # 인덱스 값들을 가지고 있는
    # 배열이다.(누적된다.)
    sequences_input_index = []
    # 하나의 인코딩 되는 문장의
    # 길이를 가지고 있다.(누적된다.)
    sequences_length = []
    # 형태소 토크나이징 사용 유무
    if tokenize_as_morph:
        value = prepro_like_morphlized(value)

    # 한줄씩 불어온다.
    # value는 질문 리스트 [질문 문장1, 질문 문장2,...] 
    # sequence 는 질문 문장 1개
    for sequence in value:
        # FILTERS = "([~.,!?\"':;)(])"
        # 정규화를 사용하여 필터에 들어 있는
        # 값들을 "" 으로 치환 한다.
        sequence = re.sub(CHANGE_FILTER, "", sequence)
        # 하나의 문장을 인코딩 할때
        # 가지고 있기 위한 배열이다.
        sequence_index = []
        # 문장을 스페이스 단위로
        # 자르고 있다.

        # sequence.split() 은 ['나는', '밥을', '먹었다']
        for word in sequence.split():
            # 잘려진 단어들이 딕셔너리에 존재 하는지 보고
            # 그 값을 가져와 sequence_index에 추가한다.

            # *형태소 단위로 인덱스 부여한 dictionary에
            # 단어 단위로 잘라 key값을 찾아보고
            # 있으면 sequence_indxe에 인덱스 추가
            if dictionary.get(word) is not None:

                # 리스트.extend(요소1개)
                # list = [2, 9, 3]
                # list.append('a')
                # print(list)

                sequence_index.extend([dictionary[word]])
            
            # 없으면 
            # 경우 이므로 UNK를(3를) 넣어 준다.
            else:
                sequence_index.extend([dictionary[UNK]])

        # sequence_index[0] = [34,6,21312,31223,3,123]
        # 없으면 '<UNK>' = 3

        # 문장 제한 길이보다 길어질 경우 뒤에 토큰을 자르고 있다.
        
        if len(sequence_index) > MAX_SEQUENCE:
            sequence_index = sequence_index[:MAX_SEQUENCE]

        # sequcences_legth => padding까지한 문장 당 번호를 붙힘
        # 하나의 문장에 길이를 넣어주고 있다.
        sequences_length.append(len(sequence_index))

        # max_sequence_length보다 문장 길이가
        # 작다면 빈 부분에 PAD(0) = 0를 넣어준다.
        # sequence_index[0] = [34,6,21312,31223,3,123,0,0,0,0,0,0,0....] 촤대길이까지 맞쳐줌
        sequence_index += (MAX_SEQUENCE - len(sequence_index)) * [dictionary[PAD]]
        
        # * 문장 길이가 길면 자르고
        #   짧으면 PAD를 넣어준다 => 학습하기 위해 문장 길이를 맞혀준다.
        
        # 인덱스화 되어 있는 값을
        # sequences_input_index에 넣어 준다.
        sequences_input_index.append(sequence_index)


    # 인덱스화된 일반 배열을 넘파이 배열로 변경한다.
    # 이유는 텐서플로우 dataset에 넣어 주기 위한
    # 사전 작업이다.
    # 넘파이 배열에 인덱스화된 배열과
    # 그 길이를 넘겨준다.
    return np.asarray(sequences_input_index), sequences_length

# Main문에서
# dec_output_processing(outputs, char2idx, tokenize_as_morph=False)
# dec_output_processing(대답 리스트, 인덱스:형태소, tokenize_as_morph=False)
def dec_output_processing(value, dictionary, tokenize_as_morph=False):

    MAX_SEQUENCE = 25 

    # 인덱스 값들을 가지고 있는
    # 배열이다.(누적된다)
    sequences_output_index = []
    # 하나의 디코딩 입력 되는 문장의
    # 길이를 가지고 있다.(누적된다)
    sequences_length = []
    # 형태소 토크나이징 사용 유무
    if tokenize_as_morph:
        value = prepro_like_morphlized(value)
    # 한줄씩 불어온다.

    # sequence는 대답 문장 1개
    for sequence in value:
        # FILTERS = "([~.,!?\"':;)(])"
        # 정규화를 사용하여 필터에 들어 있는
        # 값들을 "" 으로 치환 한다.
        sequence = re.sub(CHANGE_FILTER, "", sequence)
        # 하나의 문장을 디코딩 할때 가지고
        # 있기 위한 배열이다.
        sequence_index = []
        # 디코딩 입력의 처음에는 START가 와야 하므로
        # 그 값을 넣어 주고 시작한다.
        # 문장에서 스페이스 단위별로 단어를 가져와서 딕셔너리의
        # 값인 인덱스를 넣어 준다.
        sequence_index = [dictionary[STD]] + [dictionary[word] if word in dictionary else dictionary[UNK] for word in sequence.split()]
        # 문장 제한 길이보다 길어질 경우 뒤에 토큰을 자르고 있다.
        if len(sequence_index) > MAX_SEQUENCE:
            sequence_index = sequence_index[:MAX_SEQUENCE]
        # 하나의 문장에 길이를 넣어주고 있다.
        sequences_length.append(len(sequence_index))
        # max_sequence_length보다 문장 길이가
        # 작다면 빈 부분에 PAD(0)를 넣어준다.
        sequence_index += (MAX_SEQUENCE - len(sequence_index)) * [dictionary[PAD]]
        # 인덱스화 되어 있는 값을
        # sequences_output_index 넣어 준다.
        sequences_output_index.append(sequence_index)
    # 인덱스화된 일반 배열을 넘파이 배열로 변경한다.
    # 이유는 텐서플로우 dataset에 넣어 주기 위한
    # 사전 작업이다.
    # 넘파이 배열에 인덱스화된 배열과 그 길이를 넘겨준다.
    return np.asarray(sequences_output_index), sequences_length

# Main문에서
# dec_target_processing(outputs, char2idx, tokenize_as_morph=False)
# dec_target_processing(대답 리스트, 형태소:인덱스 dict, tokenize_as_morph=False)
def dec_target_processing(value, dictionary, tokenize_as_morph=False):

    MAX_SEQUENCE = 25 

    # 인덱스 값들을 가지고 있는
    # 배열이다.(누적된다)
    sequences_target_index = []
    # 형태소 토크나이징 사용 유무
    if tokenize_as_morph:
        value = prepro_like_morphlized(value)
    # 한줄씩 불어온다.

    # sequence는 대답 문장 1개
    for sequence in value:
        # FILTERS = "([~.,!?\"':;)(])"
        # 정규화를 사용하여 필터에 들어 있는
        # 값들을 "" 으로 치환 한다.
        sequence = re.sub(CHANGE_FILTER, "", sequence)
        # 문장에서 스페이스 단위별로 단어를 가져와서
        # 딕셔너리의 값인 인덱스를 넣어 준다.
        # 디코딩 출력의 마지막에 END를 넣어 준다.
        sequence_index = [dictionary[word] if word in dictionary else dictionary[UNK] for word in sequence.split()]
        # 문장 제한 길이보다 길어질 경우 뒤에 토큰을 자르고 있다.
        # 그리고 END 토큰을 넣어 준다
        if len(sequence_index) >= MAX_SEQUENCE:
            sequence_index = sequence_index[:MAX_SEQUENCE - 1] + [dictionary[END]]
        else:
            sequence_index += [dictionary[END]]
        # max_sequence_length보다 문장 길이가
        # 작다면 빈 부분에 PAD(0)를 넣어준다.
        sequence_index += (MAX_SEQUENCE - len(sequence_index)) * [dictionary[PAD]]
        # 인덱스화 되어 있는 값을
        # sequences_target_index에 넣어 준다.
        sequences_target_index.append(sequence_index)
    # 인덱스화된 일반 배열을 넘파이 배열로 변경한다.
    # 이유는 텐서플로우 dataset에 넣어 주기 위한 사전 작업이다.
    # 넘파이 배열에 인덱스화된 배열과 그 길이를 넘겨준다.
    return np.asarray(sequences_target_index)

# 질문 텍스트-> 벡터화 리스트, 질문 원본 인덱스 갯수 리스트(PADDING 전)
# <PAD> 만 추가함
index_inputs, input_seq_len = enc_processing(inputs, char2idx, tokenize_as_morph=False)

# 대답 텍스트-> 벡터화 리스트, 대답 원본 인덱스 갯수 리스트(PADDING 전)
# 대답 데이터는  [[<STD>,숫자,숫자..],[<STD>,숫자1,숫자2...]...]
index_outputs, output_seq_len = dec_output_processing(outputs, char2idx, tokenize_as_morph=False)

# 대답 텍스트-> [[숫자1,숫자2....<END>,<PAD>,<PAD>]]로 변형
index_targets = dec_target_processing(outputs, char2idx, tokenize_as_morph=False)

In [51]:
inputs[13], outputs[13]

('가끔은 혼자인게 좋다', '혼자를 즐기세요.')

In [52]:
# '가끔은 혼자인게 좋다' => 인덱스화 
index_inputs[13], input_seq_len[13]

(array([ 4595, 11035, 11003,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0]), 3)

In [53]:
# '혼자를 즐기세요.' => <STD> + 인덱스화 + <PAD>...

index_outputs[13], output_seq_len[13]

(array([   1, 4557, 6503,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0]), 3)

In [54]:
# '혼자를 즐기세요.' => 인덱스화 + <END> + <PAD>....

index_targets[13]

array([4557, 6503,    2,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0])

In [55]:
data_configs = {}
data_configs['char2idx'] = char2idx
data_configs['idx2char'] = idx2char
data_configs['vocab_size'] = vocab_size
data_configs['pad_symbol'] = PAD
data_configs['std_symbol'] = STD
data_configs['end_symbol'] = END
data_configs['unk_symbol'] = UNK

In [ ]:
char2idx , idx2char , vocab_size

In [57]:
DATA_IN_PATH = '/content/drive/MyDrive/새싹_인공지능SW교육/프로젝트/새싹_최종프로젝트/chatbot_s2s/data/'
TRAIN_INPUTS = 'train_inputs.npy'
TRAIN_OUTPUTS = 'train_outputs.npy'
TRAIN_TARGETS = 'train_targets.npy'
DATA_CONFIGS = 'data_configs.json'

In [58]:
np.save(open(DATA_IN_PATH + TRAIN_INPUTS, 'wb'), index_inputs)
np.save(open(DATA_IN_PATH + TRAIN_OUTPUTS , 'wb'), index_outputs)
np.save(open(DATA_IN_PATH + TRAIN_TARGETS , 'wb'), index_targets)

json.dump(data_configs, open(DATA_IN_PATH + DATA_CONFIGS, 'w'))

# 데이터 전처리 끝!

# 시퀀스 투 시퀀스 모델 만들기!

In [62]:
import tensorflow as tf
import numpy as np
import os

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt

In [63]:
def plot_graphs(history, string):
    plt.plot(history.history[string])
    plt.plot(history.history['val_'+string], '')
    plt.xlabel("Epochs")
    plt.ylabel(string)
    plt.legend([string, 'val_'+string])
    plt.show()

In [76]:
DATA_IN_PATH = '/content/drive/MyDrive/새싹_인공지능SW교육/프로젝트/새싹_최종프로젝트/chatbot_s2s/data/'
DATA_OUT_PATH = '/content/drive/MyDrive/새싹_인공지능SW교육/프로젝트/새싹_최종프로젝트/chatbot_s2s/models/'
TRAIN_INPUTS = 'train_inputs.npy'
TRAIN_OUTPUTS = 'train_outputs.npy'
TRAIN_TARGETS = 'train_targets.npy'
DATA_CONFIGS = 'data_configs.json'

In [77]:
SEED_NUM = 1234
tf.random.set_seed(SEED_NUM)

In [78]:
index_inputs = np.load(open(DATA_IN_PATH + TRAIN_INPUTS, 'rb'))
index_outputs = np.load(open(DATA_IN_PATH + TRAIN_OUTPUTS , 'rb'))
index_targets = np.load(open(DATA_IN_PATH + TRAIN_TARGETS , 'rb'))
prepro_configs = json.load(open(DATA_IN_PATH + DATA_CONFIGS, 'r'))

In [79]:
# Show length
print(len(index_inputs),  len(index_outputs), len(index_targets))

11823 11823 11823


In [80]:
MODEL_NAME = 'seq2seq_kor'
BATCH_SIZE = 2
MAX_SEQUENCE = 25
EPOCH = 30
UNITS = 1024
EMBEDDING_DIM = 256
VALIDATION_SPLIT = 0.1 

char2idx = prepro_configs['char2idx']
idx2char = prepro_configs['idx2char']
std_index = prepro_configs['std_symbol']
end_index = prepro_configs['end_symbol']
vocab_size = prepro_configs['vocab_size']

In [81]:
class Encoder(tf.keras.layers.Layer):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.vocab_size = vocab_size 
        self.embedding_dim = embedding_dim          
        
        self.embedding = tf.keras.layers.Embedding(self.vocab_size, self.embedding_dim)
        self.gru = tf.keras.layers.GRU(self.enc_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')

    def call(self, x, hidden):
        x = self.embedding(x)
        output, state = self.gru(x, initial_state = hidden)
        return output, state

    def initialize_hidden_state(self, inp):
        return tf.zeros((tf.shape(inp)[0], self.enc_units))

In [82]:
class BahdanauAttention(tf.keras.layers.Layer):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, query, values):
        hidden_with_time_axis = tf.expand_dims(query, 1)

        score = self.V(tf.nn.tanh(
            self.W1(values) + self.W2(hidden_with_time_axis)))

        attention_weights = tf.nn.softmax(score, axis=1)

        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, attention_weights

In [83]:
class Decoder(tf.keras.layers.Layer):
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
        super(Decoder, self).__init__()
        
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.vocab_size = vocab_size 
        self.embedding_dim = embedding_dim  
        
        self.embedding = tf.keras.layers.Embedding(self.vocab_size, self.embedding_dim)
        self.gru = tf.keras.layers.GRU(self.dec_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')
        self.fc = tf.keras.layers.Dense(self.vocab_size)

        self.attention = BahdanauAttention(self.dec_units)
        
    def call(self, x, hidden, enc_output):
        context_vector, attention_weights = self.attention(hidden, enc_output)

        x = self.embedding(x)

        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

        output, state = self.gru(x)
        output = tf.reshape(output, (-1, output.shape[2]))
            
        x = self.fc(output)
        
        return x, state, attention_weights

In [84]:
optimizer = tf.keras.optimizers.Adam()

loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='accuracy')

def loss(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    return tf.reduce_mean(loss_)

def accuracy(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    mask = tf.expand_dims(tf.cast(mask, dtype=pred.dtype), axis=-1)
    pred *= mask    
    acc = train_accuracy(real, pred)

    return tf.reduce_mean(acc)

In [85]:
class seq2seq(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, dec_units, batch_sz, end_token_idx=2):    
        super(seq2seq, self).__init__()
        self.end_token_idx = end_token_idx
        self.encoder = Encoder(vocab_size, embedding_dim, enc_units, batch_sz) 
        self.decoder = Decoder(vocab_size, embedding_dim, dec_units, batch_sz) 

    def call(self, x):
        inp, tar = x
        
        enc_hidden = self.encoder.initialize_hidden_state(inp)
        enc_output, enc_hidden = self.encoder(inp, enc_hidden)

        dec_hidden = enc_hidden

        predict_tokens = list()
        for t in range(0, tar.shape[1]):
            dec_input = tf.dtypes.cast(tf.expand_dims(tar[:, t], 1), tf.float32) 
            predictions, dec_hidden, _ = self.decoder(dec_input, dec_hidden, enc_output)
            predict_tokens.append(tf.dtypes.cast(predictions, tf.float32))   
        return tf.stack(predict_tokens, axis=1)
    
    def inference(self, x):
        inp  = x

        enc_hidden = self.encoder.initialize_hidden_state(inp)
        enc_output, enc_hidden = self.encoder(inp, enc_hidden)

        dec_hidden = enc_hidden
        
        dec_input = tf.expand_dims([char2idx[std_index]], 1)
        
        predict_tokens = list()
        for t in range(0, MAX_SEQUENCE):
            predictions, dec_hidden, _ = self.decoder(dec_input, dec_hidden, enc_output)
            predict_token = tf.argmax(predictions[0])
            
            if predict_token == self.end_token_idx:
                break
            
            predict_tokens.append(predict_token)
            dec_input = tf.dtypes.cast(tf.expand_dims([predict_token], 0), tf.float32)   
            
        return tf.stack(predict_tokens, axis=0).numpy()

In [86]:
model = seq2seq(vocab_size, EMBEDDING_DIM, UNITS, UNITS, BATCH_SIZE, char2idx[end_index])
model.compile(loss=loss, optimizer=tf.keras.optimizers.Adam(1e-3), metrics=[accuracy])
#model.run_eagerly = True

In [ ]:
PATH = DATA_OUT_PATH + MODEL_NAME
if not(os.path.isdir(PATH)):
        os.makedirs(os.path.join(PATH))
        
checkpoint_path = DATA_OUT_PATH + MODEL_NAME + '/weights.h5'
    
cp_callback = ModelCheckpoint(
    checkpoint_path, monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=True)

earlystop_callback = EarlyStopping(monitor='val_accuracy', min_delta=0.0001, patience=10)

history = model.fit([index_inputs, index_outputs], index_targets,
                    batch_size=BATCH_SIZE, epochs=EPOCH,
                    validation_split=VALIDATION_SPLIT, callbacks=[earlystop_callback, cp_callback])

Epoch 1/30
3942/5320 [=====================>........] - ETA: 3:15 - loss: 1.2673 - accuracy: 0.8569